<a href="https://colab.research.google.com/github/mavropulokn/pandasHw/blob/main/pandasHw.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from datetime import datetime

import numpy as np
import pandas as pd
from IPython.display import display


class FeatureGenerator:
    def __init__(self, df_train_path, df_questions_path, df_lectures_path, skip_blank_lines=True, nrows=None):
        pd.options.display.max_columns = 100
        self.df_train = pd.read_csv(df_train_path, skip_blank_lines=skip_blank_lines, nrows=nrows)
        self.df_lectures = pd.read_csv(df_lectures_path, skip_blank_lines=skip_blank_lines, nrows=nrows)
        self.df_questions = pd.read_csv(df_questions_path, skip_blank_lines=skip_blank_lines, nrows=nrows)
        self.df_merged = None

    def generate(self):
        self.clear_data()
        self.merge_atomise_dataframes()
        self.analyze_create_features()
        self.select_features()

    def clear_data(self):
        print(datetime.utcnow(), ": <-- Очистка данных -->")

        """По условию значения df_train prior_question_elapsed_time, user_answer, answered_correctly и
              prior_question_elapsed_time отсутствуют для лекций, поэтому очистим лишь прочие строки df_train"""
        self.df_train = self.df_train[self.df_train['row_id'].notna()
                                      & self.df_train['timestamp'].notna()
                                      & self.df_train['user_id'].notna()
                                      & self.df_train['content_id'].notna()
                                      & self.df_train['content_type_id'].notna()
                                      & self.df_train['task_container_id'].notna()]

        self.df_train = self.clear_data_auxiliary(self.df_train, "df_train", False)
        self.df_questions = self.clear_data_auxiliary(self.df_questions, "df_questions")
        self.df_lectures = self.clear_data_auxiliary(self.df_lectures, "df_lectures")

        print("Выведем первые три строки и посмотрим, насколько полезны столбцы:")
        print("---------------------------------------------------------")
        display(self.df_train.head(3))
        display(self.df_questions.head(3))
        display(self.df_lectures.head(3))
        print("---------------------------------------------------------")

        """Удалим столбец type_of из df_lectures, краткое описание лекции,
              поскольку качественная обработка данных значений 
              может потребовать глубокого использования инструментов NLP,
              например лемматизации или стемматизации"""
        self.df_lectures = self.df_lectures.drop('type_of', axis=1)

        """Удалим столбец user_answer из df_train,
              поскольку нам интересны лишь результаты ответа, а тип интерактивности
              можно вывести df_train content_type_id"""
        self.df_train = self.df_train.drop('user_answer', axis=1)

        """Удалим столбец correct_answer из df_questions,"
              поскольку верный ответ или нет уже известно из df_train answered_correctly"""
        self.df_questions = self.df_questions.drop('correct_answer', axis=1)

        print(datetime.utcnow(), ": <-- Очистка данных завершена -->")

    def merge_atomise_dataframes(self):
        print(datetime.utcnow(), ": <-- Объединение датафреймов в один и приведение к атомарности значений -->")

        print("Выведем типы для train dataframe: ", self.df_train.dtypes)

        """Объединим датафреймы, добавив к train, преобразуем к атомарности значения столбцов."""
        self.df_merged = self.df_train.merge(self.df_questions, how="left", left_on='content_id',
                                             right_on="question_id")
        self.df_merged = self.df_merged.drop('question_id', axis=1)
        self.df_merged = self.df_merged.rename(columns={'tags': 'tag_question'})
        self.df_train = None
        self.df_questions = None

        self.df_merged = self.df_merged.merge(self.df_lectures, how="left", left_on='content_id',
                                              right_on="lecture_id", suffixes=("", "_lecture"))
        self.df_merged = self.df_merged.drop('lecture_id', axis=1)
        self.df_merged = self.df_merged.rename(columns={'tag': 'tag_lecture'})

        self.df_lectures = None

        self.df_merged['tag_question'] = self.df_merged['tag_question'].str.split(" ")
        self.df_merged = self.df_merged.explode('tag_question')  # приведем значения к атомарным

        print(datetime.utcnow(),
              ": <-- Объединение датафреймов в один и приведение к атомарности значений завершено -->")

    def analyze_create_features(self):
        print(datetime.utcnow(), ": <-- Анализ объединенного датафрейма и создание фичей -->")
        print("Выведем типы для объединенного dataframe: ", self.df_merged.dtypes)
        print("Выведем размерность для объединенного dataframe: ", self.df_merged.shape)
        print("Выведем описательную статистику для объединенного dataframe: ", self.df_merged.describe())

        print(self.df_merged)
        print(datetime.utcnow(), ": <-- Анализ объединенного датафрейма и создание фичей -->")

    def select_features(self):
        pass

    @staticmethod
    def clear_data_auxiliary(data_frame, data_frame_name, dropna=True):
        print(datetime.utcnow(), ": Очищаем данные %s .." % data_frame_name)
        if (dropna):
            print("измерения %s до dropna():" % data_frame_name, data_frame.shape)
            data_frame = data_frame.dropna()
        print("измерения %s до удаления пустых строк:" % data_frame_name, data_frame.shape)
        data_frame = data_frame.replace(r'^\s*$', np.nan, regex=True)
        print("измерения %s после очистки:" % data_frame_name, data_frame.shape)
        return data_frame


In [ ]:
from mipt.magistr.sem1.mipt_python.pandas_hw.FeatureGenerator import FeatureGenerator

if __name__ == '__main__':
    # feature_generator = FeatureGenerator(r"C:\\Users\\kM\\Downloads\\riiid-test-answer-prediction\\train.csv")
    generator = FeatureGenerator(r"C:\\Users\\kM\\Downloads\\riiid-test-answer-prediction\\train.csv",
                                 r"C:\\Users\\kM\\Downloads\\riiid-test-answer-prediction\\questions.csv",
                                 r"C:\\Users\\kM\\Downloads\\riiid-test-answer-prediction\\lectures.csv", nrows=300000)

    generator.generate()